In [1]:
rm(list = ls())
library(tidyverse)
library(data.table)
library(org.Hs.eg.db)
library(clusterProfiler)
library(biomaRt)
library(enrichplot)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.3     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.4     ✔ forcats 1.0.0
Warning message:
“package ‘ggplot2’ was built under R version 4.2.3”
Warning message:
“package ‘tibble’ was built under R version 4.2.3”
Warning message:
“package ‘tidyr’ was built under R version 4.2.3”
Warning message:
“package ‘readr’ was built under R version 4.2.3”
Warning message:
“package ‘purrr’ was built under R version 4.2.3”
Warning message:
“package ‘dplyr’ was built under R version 4.2.3”
Warning message:
“package ‘stringr’ was built under R version 4.2.3”
Warning message:
“package ‘forcats’ was built under R version 4.2.3”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Warning message:
“package ‘data.table’ was built under R version 4.2.3”

A

In [3]:
go = read.table('./annotation/go.txt',sep = ' ',header =T,quote = ""
               )

In [4]:
colnames(go) = c('term','gene')

ERROR: Error in names(x) <- value: 'names' attribute [2] must be the same length as the vector [1]


In [34]:
allgene = read.table('./3.enrichment/DEGsid/ALL_KO-CLPvsWT-CLP.txt')

In [35]:
head(allgene,2)

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
WT-NC8.dna_NODE_548486_length_134_cov_4.714286_1,2.898979,-0.3884842,0.8377161,-0.4637421,0.6428325,0.9999922
WT-NC8.dna_NODE_548322_length_140_cov_3.076923_1,14.213618,0.3396720,0.5809086,0.5847254,0.5587324,0.9999922


In [36]:
diffgene = allgene %>% rownames_to_column(.,'gene')

In [37]:
data_all_sort <- diffgene %>%
 arrange(desc(log2FoldChange))

In [38]:
geneList = data_all_sort$log2FoldChange 
names(geneList) <- data_all_sort$gene 
head(geneList)

KO-NC4.dna_NODE_13878_length_2755_cov_7.857306_3 
                                           7.554193 
 KO-CLP2.dna_NODE_10_length_364738_cov_4.928011_187 
                                           7.483795 
KO-CLP2.dna_NODE_119_length_147657_cov_6.542276_107 
                                           7.429685 
 KO-CLP2.dna_NODE_49_length_219554_cov_5.846423_120 
                                           7.407996 
 KO-CLP2.dna_NODE_10_length_364738_cov_4.928011_108 
                                           7.380601 
 KO-CLP2.dna_NODE_102_length_157081_cov_5.174593_80 
                                           7.308249

In [43]:
length(geneList)

[1] 214164

In [96]:
dim(go)

[1] 1791435       2

In [100]:
gsea <- GSEA(geneList,
            TERM2GENE = go,
            pvalueCutoff = 1,minGSSize = 10,maxGSSize = 5000,eps =0) 

preparing geneSet collections...

GSEA analysis...

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
leading edge analysis...

done...



In [101]:
dim(gsea@result)

[1] 1008   11

In [102]:
write.csv(as.data.frame(gsea@result),file = './GSEA/gsea-go.txt')

In [62]:
kegg = read.table('./annotation/out.emapper.kegg.txt',header = F,sep = '\t')

In [64]:
kegg = kegg[,c(3,1)]

In [65]:
colnames(kegg) = c('term','gene')

In [103]:
gsea <- GSEA(geneList,
            TERM2GENE = kegg,
            pvalueCutoff = 1,minGSSize = 10,maxGSSize = 6000,eps =0) 

preparing geneSet collections...

GSEA analysis...

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
leading edge analysis...

done...



In [104]:
write.csv(as.data.frame(gsea@result),file = './GSEA/gsea-kegg.txt')